# Data Dictionary:

race_id: str
>Identifier for each race. The first race of the season will be 1. The last race will be 819. 

race: str
>The race number of each racing day.

date: time
>Date of the race.

place: str
>The order of horse complete the race track

horse_id: str
>Identifier for each horse

horse_no: str
>Number of the horse for that race

horse: str
>Name of the horse

jockey: str
>Name of the jockey

trainer: str
>Name of the trainer

actual_weight: float
>Weight that added on the horse

declared_horse_weight: float
>Weight of the horse

draw: str
>The stall where the horse started from.

lbw: float
>Distance between the horse and the winner when winner pass the rush line

running_position: array
>Place of the horse pass different check points

win_odds: float
>Ration of beting this horse if the horse win the 1st place

class: str
>The level/grade of the horse. If there is 'Group' in class, it is a international standard race.

going: str
>Condition of race track

track: str
>Field type of the race.Turf or mud (all weather track).

prize: int
>Prize for winner

location: str
>Location of the racecourse

distance_m: int
>Distance for this race

finish_time: time
>Time to finish the race

finish_time_s: float
>Time to finish the race in second

## Please use the follow dictionary for read_csv
```
types_dict = {'race_id': int, 'race': int, 'date':str, 'place':int, 'horse_id':str, 'horse_no':str,
    'horse':str, 'jockey':str, 'trainer':str, 'actual_weight':float, 'declared_horse_weight':float,
    'draw':int, 'lbw':float, 'running_position': str,'win_odds':float, 'class':str, 'going':str,
    'track':str, 'prize':int, 'location':str, 'distance_m':int, 'finish_time':str, 'finish_time_s':float}
parse_dates = ['date', 'finish_time']
raw=pd.read_csv('../data/race_2016_to_2020.csv',dtype=types_dict,parse_dates=parse_dates)
```

In [1]:
import pandas as pd
import numpy as np
import re
import statistics

In [2]:
raw = pd.read_csv('../data/raw/all_season.csv')
raw.head()

,place,horse_no,horse,jockey,trainer,actual_weight,declared_horse_weight,draw,lbw,running_position,finish_time,win_odds,race,class,going,turf,prize,location,date
0,1,10.0,AMAZING ALWAYS(N432),K Teetan,D E Ferraris,125,1126,9,-,6 ...,1:10.37,26,1(1),Class 5 - 1200M - (40-15),GOOD,"TURF - ""A"" Course","HK$ 550,000",Sha Tin,2013/09/08
1,2,9.0,FLYING SUPREME(K226),W M Lai,T K Ng,127,1074,11,1/2,4 ...,1:10.47,14,1(1),Class 5 - 1200M - (40-15),GOOD,"TURF - ""A"" Course","HK$ 550,000",Sha Tin,2013/09/08
2,3,11.0,COSMO ONE(M410),D Whyte,Y S Tsui,123,1030,13,1-3/4,7 ...,1:10.66,4.5,1(1),Class 5 - 1200M - (40-15),GOOD,"TURF - ""A"" Course","HK$ 550,000",Sha Tin,2013/09/08
3,4,6.0,SOUTHERN SPRINGS(P139),K C Leung,C S Shum,126,1107,10,2-1/4,2 ...,1:10.71,31,1(1),Class 5 - 1200M - (40-15),GOOD,"TURF - ""A"" Course","HK$ 550,000",Sha Tin,2013/09/08
4,5,5.0,BERNARD'S CHOICE(N156),Z Purton,P O'Sullivan,128,1044,6,2-1/4,8 ...,1:10.72,4.3,1(1),Class 5 - 1200M - (40-15),GOOD,"TURF - ""A"" Course","HK$ 550,000",Sha Tin,2013/09/08


In [3]:
raw.shape,raw.duplicated().any()

((74165, 19), False)

In [4]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74165 entries, 0 to 74164
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   place                  74165 non-null  object 
 1   horse_no               73344 non-null  float64
 2   horse                  74165 non-null  object 
 3   jockey                 74165 non-null  object 
 4   trainer                74165 non-null  object 
 5   actual_weight          74165 non-null  int64  
 6   declared_horse_weight  74165 non-null  object 
 7   draw                   74165 non-null  object 
 8   lbw                    74165 non-null  object 
 9   running_position       74165 non-null  object 
 10  finish_time            74165 non-null  object 
 11  win_odds               74165 non-null  object 
 12  race                   74165 non-null  object 
 13  class                  74165 non-null  object 
 14  going                  74165 non-null  object 
 15  tu

In [5]:
#Drop samples with place has no digits
#Change places with DH to digit only
#reset index of dataframe
df_place=raw.copy()
df_place=df_place.drop_duplicates()
df_place=df_place[(df_place['place']!='WV')&(df_place['place']!='WV-A')]
df_place=df_place[(df_place['place']!='WX')&(df_place['place']!='WX-A')&(df_place['place']!='WXNR')]
df_place=df_place[(df_place['place']!='PU')&(df_place['place']!='UR')&(df_place['place']!='DNF')&(df_place['place']!='TNP')]
df_place=df_place[(df_place['place']!='FE')&(df_place['place']!='DISQ')]
df_place['place']=df_place['place'].str.extract(r'(\d+)')
df_place=df_place.reset_index(drop=True)
#Change horse no. to int
df_place['horse_no']=df_place['horse_no'].astype(int).astype(str)

In [6]:
#split horse name into house name and horse id
df_horse = df_place.copy()
df_horse['horse_id']=df_horse['horse'].str.extract(r'\((.+)\)')
df_horse['horse']=df_horse['horse'].str.extract(r'^(.+)\(')

#change actual weight into float
df_horse['actual_weight']=df_horse['actual_weight'].astype(float)

#change declared horse weight into float
df_horse['declared_horse_weight']=df_horse['declared_horse_weight'].astype(float)

In [7]:
#change lbw into float
df_lbw=df_horse.copy()
df_lbw.loc[df_lbw['lbw']=='-','lbw']='0'

def lbw_to_float(lbw_string):
    if lbw_string=='N':
        return 0.33
    elif lbw_string=='SH':
        return 0.1
    elif lbw_string=='HD':
        return 0.25
    elif lbw_string=='NOSE':
        return 0.02
    elif lbw_string=='ML':
        return 150
    nums = [int(i) for i in re.findall('\d+',lbw_string)]
    if len(nums)==1:
        return nums[0]
    elif len(nums)==2:
        return nums[0]/nums[1]
    elif len(nums)==3:
        return nums[0]+nums[1]/nums[2]
    else:
        return 999999

df_lbw['lbw']=df_lbw['lbw'].apply(lbw_to_float)

#change running position into array
df_lbw['running_position']=df_lbw['running_position'].str.findall(r'(\d+)')

In [8]:
#change finish_time into second with type float
df_time=df_lbw.copy()
def get_sec(finish_time):
    result=[float(i) for i in finish_time.split(':')]
    return result[0]*60+result[1]
df_time['finish_time_s']=df_time['finish_time'].apply(get_sec)
df_time['finish_time']=pd.to_datetime(df_time['finish_time'],format='%M:%S.%f')
df_time['win_odds']=df_time['win_odds'].astype(float)

In [9]:
#split race and race_id
df_class=df_time.copy()
df_class['race_id']=df_class['race'].str.extract(r'\((\d+)')
df_class['race']=df_class['race'].str.extract(r'(\d+)\(')

#extract number and class from original class data
def get_level(race_class):
    result=race_class.split('-')
    return result[0].strip()

def get_distance(race_class):
    result=race_class.split('-')
    return int(result[1].strip()[:-1])

df_class['distance_m']=df_class['class'].apply(get_distance)
df_class['class']=df_class['class'].apply(get_level)

In [10]:
#extract digit from prize
#transform date to datetime type
df_prize=df_class.copy()
df_prize['prize']=df_prize['prize'].str.replace(r'[^0-9]', "").astype(int)
df_prize['date']=pd.to_datetime(df_prize['date'])

In [11]:
#rename column turf to track 
#reorder the columns
df_eda=df_prize.copy()
df_eda['track']=df_eda['turf']
df_eda=df_eda.drop(columns='turf',axis=1)
df_eda=df_eda[['race_id', 'race', 'date', 'place', 'horse_id', 'horse_no', 'horse', 'jockey', 'trainer', 'actual_weight',
       'declared_horse_weight', 'draw', 'lbw', 'running_position', 'win_odds', 'class', 'going', 'track',
       'prize', 'location', 'distance_m', 'finish_time', 'finish_time_s']]

In [12]:
df_eda

,race_id,race,date,place,horse_id,horse_no,horse,jockey,trainer,actual_weight,...,running_position,win_odds,class,going,track,prize,location,distance_m,finish_time,finish_time_s
0,1,1,2013-09-08,1,N432,10,AMAZING ALWAYS,K Teetan,D E Ferraris,125.0,...,"[6, 6, 1]",26.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.370,70.37
1,1,1,2013-09-08,2,K226,9,FLYING SUPREME,W M Lai,T K Ng,127.0,...,"[4, 5, 2]",14.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.470,70.47
2,1,1,2013-09-08,3,M410,11,COSMO ONE,D Whyte,Y S Tsui,123.0,...,"[7, 7, 3]",4.5,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.660,70.66
3,1,1,2013-09-08,4,P139,6,SOUTHERN SPRINGS,K C Leung,C S Shum,126.0,...,"[2, 1, 4]",31.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.710,70.71
4,1,1,2013-09-08,5,N156,5,BERNARD'S CHOICE,Z Purton,P O'Sullivan,128.0,...,"[8, 9, 5]",4.3,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.720,70.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72508,325,9,2021-01-06,8,B026,5,LONDON HALL,A Badel,D J Hall,129.0,...,"[8, 9, 9, 12, 8]",8.4,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:50.920,110.92
72509,325,9,2021-01-06,9,D025,3,ARROGANT,C Schofield,C Fownes,130.0,...,"[9, 12, 12, 10, 9]",19.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:50.990,110.99
72510,325,9,2021-01-06,10,C269,12,DIVINE UNICORN,V Borges,W Y So,113.0,...,"[7, 10, 11, 11, 10]",44.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:51.180,111.18
72511,325,9,2021-01-06,11,C450,6,BLASTOISE,A Hamelin,D E Ferraris,128.0,...,"[4, 3, 2, 3, 11]",47.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:51.520,111.52


In [13]:
df_eda.to_csv('../data/race_eda.csv',index=False)

In [14]:
types_dict = {'race_id': int, 'race': int, 'date':str, 'place':int, 'horse_id':str, 'horse_no':str,
              'horse':str, 'jockey':str, 'trainer':str, 'actual_weight':float, 'declared_horse_weight':float,
              'draw':int, 'lbw':float, 'running_position': str,'win_odds':float, 'class':str, 'going':str,
              'track':str, 'prize':int, 'location':str, 'distance_m':int, 'finish_time':str, 'finish_time_s':float}
parse_dates = ['date', 'finish_time']

raw=pd.read_csv('../data/race_eda.csv',dtype=types_dict,parse_dates=parse_dates)

In [15]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72513 entries, 0 to 72512
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   race_id                72513 non-null  int64         
 1   race                   72513 non-null  int64         
 2   date                   72513 non-null  datetime64[ns]
 3   place                  72513 non-null  int64         
 4   horse_id               72513 non-null  object        
 5   horse_no               72513 non-null  object        
 6   horse                  72513 non-null  object        
 7   jockey                 72513 non-null  object        
 8   trainer                72513 non-null  object        
 9   actual_weight          72513 non-null  float64       
 10  declared_horse_weight  72513 non-null  float64       
 11  draw                   72513 non-null  int64         
 12  lbw                    72513 non-null  float64       
 13  r

In [16]:
raw

,race_id,race,date,place,horse_id,horse_no,horse,jockey,trainer,actual_weight,...,running_position,win_odds,class,going,track,prize,location,distance_m,finish_time,finish_time_s
0,1,1,2013-09-08,1,N432,10,AMAZING ALWAYS,K Teetan,D E Ferraris,125.0,...,"['6', '6', '1']",26.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.370,70.37
1,1,1,2013-09-08,2,K226,9,FLYING SUPREME,W M Lai,T K Ng,127.0,...,"['4', '5', '2']",14.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.470,70.47
2,1,1,2013-09-08,3,M410,11,COSMO ONE,D Whyte,Y S Tsui,123.0,...,"['7', '7', '3']",4.5,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.660,70.66
3,1,1,2013-09-08,4,P139,6,SOUTHERN SPRINGS,K C Leung,C S Shum,126.0,...,"['2', '1', '4']",31.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.710,70.71
4,1,1,2013-09-08,5,N156,5,BERNARD'S CHOICE,Z Purton,P O'Sullivan,128.0,...,"['8', '9', '5']",4.3,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,1900-01-01 00:01:10.720,70.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72508,325,9,2021-01-06,8,B026,5,LONDON HALL,A Badel,D J Hall,129.0,...,"['8', '9', '9', '12', '8']",8.4,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:50.920,110.92
72509,325,9,2021-01-06,9,D025,3,ARROGANT,C Schofield,C Fownes,130.0,...,"['9', '12', '12', '10', '9']",19.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:50.990,110.99
72510,325,9,2021-01-06,10,C269,12,DIVINE UNICORN,V Borges,W Y So,113.0,...,"['7', '10', '11', '11', '10']",44.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:51.180,111.18
72511,325,9,2021-01-06,11,C450,6,BLASTOISE,A Hamelin,D E Ferraris,128.0,...,"['4', '3', '2', '3', '11']",47.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,1900-01-01 00:01:51.520,111.52


In [17]:
raw.shape

(72513, 23)

In [18]:
def tran_running_position_to_list(r_pos):
    r_pos = r_pos.strip("[]")
    str_list = re.findall('(\d+)',r_pos)
    result = [int(s) for s in str_list]
    return result

def get_season(race_date):
    if race_date.month<9:
        season=str(race_date.year-1)+'/'+str(race_date.year)
    else:
        season=str(race_date.year)+'/'+str(race_date.year+1)
    return season

In [19]:
#further data cleaning
df=raw.copy()

# get season for each record
df['season'] = df['date'].apply(get_season)

# rest day = day difference between current race and last race,fillna with 45 days for summer holidays
df['rest_day']=df.groupby('horse_id')['date'].diff().apply(lambda x: x.days).fillna(45)

# generate last running position of horse and drop running position, as running position leak current result
df['running_position'] = df['running_position'].apply(tran_running_position_to_list)
df['last_running_position'] = df.groupby(['horse_id'])['running_position'].transform(lambda x:x.shift()).apply(lambda x: x if isinstance(x,list) else [0])
df['last_running_position'] = df['last_running_position'].apply(lambda x: statistics.mean(x))

# generate last place 
df['last_place'] = df.groupby(['horse_id'])['place'].transform(lambda x:x.shift()).apply(lambda x: 0. if np.isnan(x) else x)

#generate last lbw
df['last_lbw'] = df.groupby(['horse_id'])['lbw'].transform(lambda x:x.shift()).apply(lambda x: 0. if np.isnan(x) else x)

# generate last declared_horse_weight
df['last_actual_weight'] = df.groupby(['horse_id'])['actual_weight'].transform(lambda x:x.shift()).apply(lambda x: 0. if np.isnan(x) else x)

df.columns

Index(['race_id', 'race', 'date', 'place', 'horse_id', 'horse_no', 'horse',
       'jockey', 'trainer', 'actual_weight', 'declared_horse_weight', 'draw',
       'lbw', 'running_position', 'win_odds', 'class', 'going', 'track',
       'prize', 'location', 'distance_m', 'finish_time', 'finish_time_s',
       'season', 'rest_day', 'last_running_position', 'last_place', 'last_lbw',
       'last_actual_weight'],
      dtype='object')

In [20]:
df_ml=df[['season', 'race_id', 'race', 'date', 'place', 'last_place', 'horse_id', 'horse_no', 'horse',
        'jockey', 'trainer', 'actual_weight', 'last_actual_weight', 'declared_horse_weight', 'draw',
        'last_lbw', 'last_running_position','win_odds', 'class', 'going', 'track','prize', 'location',
        'distance_m', 'rest_day', 'finish_time_s']]
df_ml

,season,race_id,race,date,place,last_place,horse_id,horse_no,horse,jockey,...,last_running_position,win_odds,class,going,track,prize,location,distance_m,rest_day,finish_time_s
0,2013/2014,1,1,2013-09-08,1,0.0,N432,10,AMAZING ALWAYS,K Teetan,...,0.00,26.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,45.0,70.37
1,2013/2014,1,1,2013-09-08,2,0.0,K226,9,FLYING SUPREME,W M Lai,...,0.00,14.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,45.0,70.47
2,2013/2014,1,1,2013-09-08,3,0.0,M410,11,COSMO ONE,D Whyte,...,0.00,4.5,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,45.0,70.66
3,2013/2014,1,1,2013-09-08,4,0.0,P139,6,SOUTHERN SPRINGS,K C Leung,...,0.00,31.0,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,45.0,70.71
4,2013/2014,1,1,2013-09-08,5,0.0,N156,5,BERNARD'S CHOICE,Z Purton,...,0.00,4.3,Class 5,GOOD,"TURF - ""A"" Course",550000,Sha Tin,1200,45.0,70.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72508,2020/2021,325,9,2021-01-06,8,3.0,B026,5,LONDON HALL,A Badel,...,4.60,8.4,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,66.0,110.92
72509,2020/2021,325,9,2021-01-06,9,10.0,D025,3,ARROGANT,C Schofield,...,3.00,19.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,24.0,110.99
72510,2020/2021,325,9,2021-01-06,10,14.0,C269,12,DIVINE UNICORN,V Borges,...,9.20,44.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,24.0,111.18
72511,2020/2021,325,9,2021-01-06,11,9.0,C450,6,BLASTOISE,A Hamelin,...,6.80,47.0,Class 3,GOOD,"TURF - ""A"" Course",1500000,Happy Valley,1800,35.0,111.52


In [21]:
# generate jockey df with win rate of last season
# get name for all jockeys in data set, order by asc
df_jockey=pd.DataFrame({'jockey':df['jockey'].sort_values().unique()})
# get no. of 1st,2nd,3rd,4th place in last season
def cal_pct_1st(x):
    return (x==1).sum()/len(x)*100
def cal_pct_2nd(x):
    return (x==2).sum()/len(x)*100
def cal_pct_3rd(x):
    return (x==3).sum()/len(x)*100
def cal_pct_4th(x):
    return (x==4).sum()/len(x)*100

#create dataframe containing percentage of 1st,2nd,3rd,4th place of last season
df_1st_rate=df.groupby(['season','jockey'],as_index=False)['place'].agg(cal_pct_1st).round(decimals=2)
df_2nd_rate=df.groupby(['season','jockey'],as_index=False)['place'].agg(cal_pct_2nd).round(decimals=2)
df_3rd_rate=df.groupby(['season','jockey'],as_index=False)['place'].agg(cal_pct_3rd).round(decimals=2)
df_4th_rate=df.groupby(['season','jockey'],as_index=False)['place'].agg(cal_pct_4th).round(decimals=2)
df_1st_rate.columns=['season','jockey','pct_1st']
df_2nd_rate.columns=['season','jockey','pct_2nd']
df_3rd_rate.columns=['season','jockey','pct_3rd']
df_4th_rate.columns=['season','jockey','pct_4th']

#join pct information to df_jockey dataframe
df_jockey = pd.merge(df_jockey,df_1st_rate,on='jockey')
df_jockey = pd.merge(df_jockey,df_2nd_rate,on=['jockey','season'])
df_jockey = pd.merge(df_jockey,df_3rd_rate,on=['jockey','season'])
df_jockey = pd.merge(df_jockey,df_4th_rate,on=['jockey','season'])

#get last season performance by shifting
df_jockey['pct_1st_last_j'] = df_jockey.groupby(['jockey'],as_index=False)['pct_1st'].transform(lambda x:x.shift())
df_jockey['pct_2nd_last_j'] = df_jockey.groupby(['jockey'],as_index=False)['pct_2nd'].transform(lambda x:x.shift())
df_jockey['pct_3rd_last_j'] = df_jockey.groupby(['jockey'],as_index=False)['pct_3rd'].transform(lambda x:x.shift())
df_jockey['pct_4th_last_j'] = df_jockey.groupby(['jockey'],as_index=False)['pct_4th'].transform(lambda x:x.shift())

#keep all pact columns of last season only
df_jockey=df_jockey[['jockey','season','pct_1st_last_j','pct_2nd_last_j','pct_3rd_last_j','pct_4th_last_j']]

#fill nan with 0 for the first season record
df_jockey['pct_1st_last_j'] = df_jockey['pct_1st_last_j'].apply(lambda x: 0. if np.isnan(x) else x)
df_jockey['pct_2nd_last_j'] = df_jockey['pct_2nd_last_j'].apply(lambda x: 0. if np.isnan(x) else x) 
df_jockey['pct_3rd_last_j'] = df_jockey['pct_3rd_last_j'].apply(lambda x: 0. if np.isnan(x) else x) 
df_jockey['pct_4th_last_j'] = df_jockey['pct_4th_last_j'].apply(lambda x: 0. if np.isnan(x) else x)

df_jockey

,jockey,season,pct_1st_last_j,pct_2nd_last_j,pct_3rd_last_j,pct_4th_last_j
0,A Atzeni,2013/2014,0.00,0.00,0.00,0.00
1,A Atzeni,2014/2015,0.00,0.00,0.00,0.00
2,A Atzeni,2017/2018,3.80,3.80,3.80,7.59
3,A Badel,2016/2017,0.00,0.00,0.00,0.00
4,A Badel,2017/2018,5.98,6.84,6.84,10.26
...,...,...,...,...,...,...
445,Z Purton,2016/2017,15.09,13.96,9.25,9.62
446,Z Purton,2017/2018,17.72,15.23,9.93,7.95
447,Z Purton,2018/2019,21.45,16.88,10.09,9.46
448,Z Purton,2019/2020,24.92,16.37,9.91,9.31


In [22]:
df_trainer=pd.DataFrame({'trainer':df['trainer'].sort_values().unique()})

#create dataframe containing percentage of 1st,2nd,3rd,4th place of last season
df_1st_rate=df.groupby(['season','trainer'],as_index=False)['place'].agg(cal_pct_1st).round(decimals=2)
df_2nd_rate=df.groupby(['season','trainer'],as_index=False)['place'].agg(cal_pct_2nd).round(decimals=2)
df_3rd_rate=df.groupby(['season','trainer'],as_index=False)['place'].agg(cal_pct_3rd).round(decimals=2)
df_4th_rate=df.groupby(['season','trainer'],as_index=False)['place'].agg(cal_pct_4th).round(decimals=2)
df_1st_rate.columns=['season','trainer','pct_1st']
df_2nd_rate.columns=['season','trainer','pct_2nd']
df_3rd_rate.columns=['season','trainer','pct_3rd']
df_4th_rate.columns=['season','trainer','pct_4th']

#join pct information to df_jockey dataframe
df_trainer = pd.merge(df_trainer,df_1st_rate,on='trainer')
df_trainer = pd.merge(df_trainer,df_2nd_rate,on=['trainer','season'])
df_trainer = pd.merge(df_trainer,df_3rd_rate,on=['trainer','season'])
df_trainer = pd.merge(df_trainer,df_4th_rate,on=['trainer','season'])

#get last season performance by shifting
df_trainer['pct_1st_last_t'] = df_trainer.groupby(['trainer'],as_index=False)['pct_1st'].transform(lambda x:x.shift())
df_trainer['pct_2nd_last_t'] = df_trainer.groupby(['trainer'],as_index=False)['pct_2nd'].transform(lambda x:x.shift())
df_trainer['pct_3rd_last_t'] = df_trainer.groupby(['trainer'],as_index=False)['pct_3rd'].transform(lambda x:x.shift())
df_trainer['pct_4th_last_t'] = df_trainer.groupby(['trainer'],as_index=False)['pct_4th'].transform(lambda x:x.shift())

#keep all pact columns of last season only
df_trainer=df_trainer[['trainer','season','pct_1st_last_t','pct_2nd_last_t','pct_3rd_last_t','pct_4th_last_t']]

#fill nan with 0 for the first season record
df_trainer['pct_1st_last_t'] = df_trainer['pct_1st_last_t'].apply(lambda x: 0. if np.isnan(x) else x)
df_trainer['pct_2nd_last_t'] = df_trainer['pct_2nd_last_t'].apply(lambda x: 0. if np.isnan(x) else x) 
df_trainer['pct_3rd_last_t'] = df_trainer['pct_3rd_last_t'].apply(lambda x: 0. if np.isnan(x) else x) 
df_trainer['pct_4th_last_t'] = df_trainer['pct_4th_last_t'].apply(lambda x: 0. if np.isnan(x) else x)

df_trainer

,trainer,season,pct_1st_last_t,pct_2nd_last_t,pct_3rd_last_t,pct_4th_last_t
0,A Bull,2016/2017,0.0,0.0,0.0,0.0
1,A Couetil,2017/2018,0.0,0.0,0.0,0.0
2,A Fabre,2014/2015,0.0,0.0,0.0,0.0
3,A Fabre,2015/2016,100.0,0.0,0.0,0.0
4,A Fabre,2016/2017,0.0,50.0,0.0,50.0
...,...,...,...,...,...,...
391,Y Yahagi,2018/2019,0.0,0.0,100.0,0.0
392,de Royer Dupre,2013/2014,0.0,0.0,0.0,0.0
393,de Royer Dupre,2015/2016,0.0,0.0,0.0,0.0
394,de Royer Dupre,2016/2017,0.0,0.0,100.0,0.0


In [23]:
# export race,jockey,trainer dataframe to csv file
df_ml.to_csv('../data/race.csv')
df_jockey.to_csv('../data/jockey.csv')
df_trainer.to_csv('../data/trainer.csv')

In [24]:
# join df_ml,df_jockey,df_trainer together
df_r_j=pd.merge(df_ml, df_jockey, how='left', left_on=['season','jockey'], right_on=['season','jockey'] )
df_all=pd.merge(df_r_j,df_trainer, how='left', left_on=['season','trainer'], right_on=['season','trainer'])
df_all

,season,race_id,race,date,place,last_place,horse_id,horse_no,horse,jockey,...,rest_day,finish_time_s,pct_1st_last_j,pct_2nd_last_j,pct_3rd_last_j,pct_4th_last_j,pct_1st_last_t,pct_2nd_last_t,pct_3rd_last_t,pct_4th_last_t
0,2013/2014,1,1,2013-09-08,1,0.0,N432,10,AMAZING ALWAYS,K Teetan,...,45.0,70.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2013/2014,1,1,2013-09-08,2,0.0,K226,9,FLYING SUPREME,W M Lai,...,45.0,70.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2013/2014,1,1,2013-09-08,3,0.0,M410,11,COSMO ONE,D Whyte,...,45.0,70.66,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2013/2014,1,1,2013-09-08,4,0.0,P139,6,SOUTHERN SPRINGS,K C Leung,...,45.0,70.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2013/2014,1,1,2013-09-08,5,0.0,N156,5,BERNARD'S CHOICE,Z Purton,...,45.0,70.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72508,2020/2021,325,9,2021-01-06,8,3.0,B026,5,LONDON HALL,A Badel,...,66.0,110.92,10.28,9.09,8.30,9.09,7.91,5.36,8.93,10.20
72509,2020/2021,325,9,2021-01-06,9,10.0,D025,3,ARROGANT,C Schofield,...,24.0,110.99,8.78,8.16,8.16,8.37,9.30,10.25,7.78,9.68
72510,2020/2021,325,9,2021-01-06,10,14.0,C269,12,DIVINE UNICORN,V Borges,...,24.0,111.18,6.64,7.47,7.05,6.64,7.31,6.60,7.78,7.78
72511,2020/2021,325,9,2021-01-06,11,9.0,C450,6,BLASTOISE,A Hamelin,...,35.0,111.52,7.49,7.49,7.05,10.57,4.92,6.56,8.43,6.56


In [25]:
df_all.columns

Index(['season', 'race_id', 'race', 'date', 'place', 'last_place', 'horse_id',
       'horse_no', 'horse', 'jockey', 'trainer', 'actual_weight',
       'last_actual_weight', 'declared_horse_weight', 'draw', 'last_lbw',
       'last_running_position', 'win_odds', 'class', 'going', 'track', 'prize',
       'location', 'distance_m', 'rest_day', 'finish_time_s', 'pct_1st_last_j',
       'pct_2nd_last_j', 'pct_3rd_last_j', 'pct_4th_last_j', 'pct_1st_last_t',
       'pct_2nd_last_t', 'pct_3rd_last_t', 'pct_4th_last_t'],
      dtype='object')

In [26]:
df_all.head().T

,0,1,2,3,4
season,2013/2014,2013/2014,2013/2014,2013/2014,2013/2014
race_id,1,1,1,1,1
race,1,1,1,1,1
date,2013-09-08 00:00:00,2013-09-08 00:00:00,2013-09-08 00:00:00,2013-09-08 00:00:00,2013-09-08 00:00:00
place,1,2,3,4,5
last_place,0,0,0,0,0
horse_id,N432,K226,M410,P139,N156
horse_no,10,9,11,6,5
horse,AMAZING ALWAYS,FLYING SUPREME,COSMO ONE,SOUTHERN SPRINGS,BERNARD'S CHOICE
jockey,K Teetan,W M Lai,D Whyte,K C Leung,Z Purton


In [27]:
df_train=df_all[df_all['date']<'2019-09-01']
df_test=df_all[df_all['date']>='2019-09-01']

In [29]:
df_train.shape[0], df_test.shape[0], df_train.shape[0]+df_test.shape[0]

(58366, 14147, 72513)

In [30]:
df_train.to_csv('../data/train/train.csv')
df_train

,season,race_id,race,date,place,last_place,horse_id,horse_no,horse,jockey,...,rest_day,finish_time_s,pct_1st_last_j,pct_2nd_last_j,pct_3rd_last_j,pct_4th_last_j,pct_1st_last_t,pct_2nd_last_t,pct_3rd_last_t,pct_4th_last_t
0,2013/2014,1,1,2013-09-08,1,0.0,N432,10,AMAZING ALWAYS,K Teetan,...,45.0,70.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2013/2014,1,1,2013-09-08,2,0.0,K226,9,FLYING SUPREME,W M Lai,...,45.0,70.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2013/2014,1,1,2013-09-08,3,0.0,M410,11,COSMO ONE,D Whyte,...,45.0,70.66,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2013/2014,1,1,2013-09-08,4,0.0,P139,6,SOUTHERN SPRINGS,K C Leung,...,45.0,70.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2013/2014,1,1,2013-09-08,5,0.0,N156,5,BERNARD'S CHOICE,Z Purton,...,45.0,70.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58361,2018/2019,817,11,2019-07-14,10,11.0,T375,14,SICHUAN DAR,V Borges,...,18.0,81.52,0.00,0.00,0.00,0.00,11.72,8.28,10.34,10.00
58362,2018/2019,817,11,2019-07-14,11,8.0,V077,1,JOLLY BANNER,K H Chan,...,21.0,81.63,0.00,0.00,0.00,0.00,8.45,6.52,11.11,8.70
58363,2018/2019,817,11,2019-07-14,12,6.0,B312,6,THANKS FOREVER,Z Purton,...,13.0,83.23,21.45,16.88,10.09,9.46,6.63,7.83,7.03,8.84
58364,2018/2019,817,11,2019-07-14,13,7.0,V118,8,GORGEOUS KING,T H So,...,134.0,83.57,1.92,5.77,9.62,5.77,8.85,9.29,8.63,8.41


In [31]:
df_test.to_csv('../data/test/test.csv')

In [32]:
df_test.tail().T

,72508,72509,72510,72511,72512
season,2020/2021,2020/2021,2020/2021,2020/2021,2020/2021
race_id,325,325,325,325,325
race,9,9,9,9,9
date,2021-01-06 00:00:00,2021-01-06 00:00:00,2021-01-06 00:00:00,2021-01-06 00:00:00,2021-01-06 00:00:00
place,8,9,10,11,12
last_place,3,10,14,9,4
horse_id,B026,D025,C269,C450,D203
horse_no,5,3,12,6,11
horse,LONDON HALL,ARROGANT,DIVINE UNICORN,BLASTOISE,TOYCOON
jockey,A Badel,C Schofield,V Borges,A Hamelin,K C Leung


In [33]:
df_test.head().T

,58366,58367,58368,58369,58370
season,2019/2020,2019/2020,2019/2020,2019/2020,2019/2020
race_id,1,1,1,1,1
race,1,1,1,1,1
date,2019-09-01 00:00:00,2019-09-01 00:00:00,2019-09-01 00:00:00,2019-09-01 00:00:00,2019-09-01 00:00:00
place,1,2,3,4,5
last_place,3,8,6,12,3
horse_id,C127,C035,V406,V369,V400
horse_no,3,4,7,2,1
horse,GOOD RUNNERS WAY,REGENCY GEM,PROUD SKY,LE PANACHE,GOLD VELVET
jockey,Z Purton,J Moreira,K Teetan,A Sanna,L Hewitson


In [34]:
df_train.shape,df_test.shape

((58366, 34), (14147, 34))

In [35]:
df_train.columns

Index(['season', 'race_id', 'race', 'date', 'place', 'last_place', 'horse_id',
       'horse_no', 'horse', 'jockey', 'trainer', 'actual_weight',
       'last_actual_weight', 'declared_horse_weight', 'draw', 'last_lbw',
       'last_running_position', 'win_odds', 'class', 'going', 'track', 'prize',
       'location', 'distance_m', 'rest_day', 'finish_time_s', 'pct_1st_last_j',
       'pct_2nd_last_j', 'pct_3rd_last_j', 'pct_4th_last_j', 'pct_1st_last_t',
       'pct_2nd_last_t', 'pct_3rd_last_t', 'pct_4th_last_t'],
      dtype='object')

In [36]:
df_last=df_all[df_all['date']=='2021-01-01']

In [37]:
df_last

,season,race_id,race,date,place,last_place,horse_id,horse_no,horse,jockey,...,rest_day,finish_time_s,pct_1st_last_j,pct_2nd_last_j,pct_3rd_last_j,pct_4th_last_j,pct_1st_last_t,pct_2nd_last_t,pct_3rd_last_t,pct_4th_last_t
72269,2020/2021,306,1,2021-01-01,1,0.0,E162,8,MASTER FAY,J Moreira,...,45.0,69.68,19.22,17.13,12.67,9.47,9.30,10.25,7.78,9.68
72270,2020/2021,306,1,2021-01-01,2,0.0,E170,10,TRAVEL GLORY,V Borges,...,45.0,70.09,6.64,7.47,7.05,6.64,4.85,9.44,5.61,3.83
72271,2020/2021,306,1,2021-01-01,3,9.0,C005,14,BRILLIANT,K C Leung,...,44.0,70.29,5.19,3.76,6.44,8.94,4.71,3.44,7.61,6.16
72272,2020/2021,306,1,2021-01-01,4,12.0,D451,6,JOYFUL WIN,T Piccone,...,48.0,70.31,8.89,6.67,13.33,5.56,14.02,9.00,9.21,9.21
72273,2020/2021,306,1,2021-01-01,5,3.0,C498,12,VIGOR CHAMP,Z Purton,...,26.0,70.34,20.82,14.31,14.16,7.93,8.50,4.85,6.07,8.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72402,2020/2021,316,11,2021-01-01,10,2.0,D187,14,THE ROCK,M Chadwick,...,16.0,96.19,4.71,8.04,8.24,6.47,9.30,10.25,7.78,9.68
72403,2020/2021,316,11,2021-01-01,11,8.0,D085,1,SUPER OASIS,C Y Ho,...,19.0,96.23,10.75,9.63,9.31,9.95,14.02,9.00,9.21,9.21
72404,2020/2021,316,11,2021-01-01,12,0.0,E194,7,ARTHUR'S KINGDOM,K C Leung,...,45.0,96.40,5.19,3.76,6.44,8.94,5.53,7.87,7.37,6.20
72405,2020/2021,316,11,2021-01-01,13,10.0,E189,5,THE SUMMIT,B Shinn,...,19.0,96.57,3.39,9.71,8.58,7.67,0.00,0.00,0.00,0.00


In [38]:
df_last.to_csv('../data/last.csv')